<a href="https://colab.research.google.com/github/RyanGoslenko/Data_Science/blob/main/colab/finetune_gpt_j_6B_8bit_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fine-tuning 6-Billion GPT-J in colab with LoRA and 8-bit compression

This notebook is a proof of concept for fine-tuning [GPT-J-6B](https://huggingface.co/EleutherAI/gpt-j-6B) with limited memory. A detailed explanation of how it works can be found in [this model card](https://huggingface.co/hivemind/gpt-j-6B-8bit).

In [1]:
!pip install transformers==4.14.1
!pip install bitsandbytes
!pip install datasets==1.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 32.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=21fefa6e16c02973d92e035a3f8401dac7c8534ba5520f30132e973311205f6d
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 11.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.

In [2]:
import numpy as np
import transformers
import torch
import torch.nn.functional as functional

from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from bitsandbytes.optim import Adam8bit
from tqdm.auto import tqdm


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


### Converting the model to 8 bits.

We convert EleutherAI's GPT-J-6B model to 8 bits using facebook's [bitsandbytes](https://github.com/facebookresearch/bitsandbytes) library. This reduces the model's size from 20Gb down to just 6Gb.

Note that we don't convert linear layer biases to 8 bit as they take up less that 1% of the model's weight anyway.

In [3]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias).clone()
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [4]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [5]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [6]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.75G [00:00<?, ?B/s]

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0

In [11]:
def add_adapters(model, adapter_dim=16):
    assert adapter_dim > 0

    for name, module in model.named_modules():
        if "attn" in name:
          if isinstance(module, FrozenBNBLinear):
              module.adapter = nn.Sequential(
                  nn.Linear(module.in_features, adapter_dim, bias=False),
                  nn.Linear(adapter_dim, module.out_features, bias=False),
              )
              nn.init.zeros_(module.adapter[1].weight)
          elif isinstance(module, FrozenBNBEmbedding):
              module.adapter = nn.Sequential(
                  nn.Embedding(module.num_embeddings, adapter_dim),
                  nn.Linear(adapter_dim, module.embedding_dim, bias=False),
              )
              nn.init.zeros_(module.adapter[1].weight)

        if hasattr(module, "adapter"):
            print(module)
            nn.init.zeros_(module.adapter[1].weight)
            # optional: scale adapter[0].weight by (LoRA_alpha / r)
add_adapters(gpt)
gpt.to(device)

FrozenBNBEmbedding(50400, 4096)


TypeError: ignored

In [8]:
with open('small_train.txt', 'r') as f:
    train_data = f.readlines()
with open('small_test.txt', 'r') as f:
    test_data = f.readlines()
with open('small_valid.txt', 'r') as f:
    valid_data = f.readlines()

In [16]:
def train(data, n_epochs, lr, lr_update=False, seq_length=256, weight_decay=0, verbose=False):
    optimizer = Adam8bit(gpt.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = transformers.get_linear_schedule_with_warmup(
            optimizer=optimizer,
            num_warmup_steps=int(0.05*len(data) * n_epochs),
            num_training_steps=len(data) * n_epochs)
    
    gpt.gradient_checkpointing_enable()
    
    model_best_state = {}
    train_losses = []
    train_perplexities = []
    valid_losses = []
    valid_perplexities = []
    min_valid_loss = np.inf

    for epoch in range(n_epochs):
        with torch.cuda.amp.autocast():
            # Trainning loop
            for row in tqdm(train_data):
                if len(row) <= 1:
                    continue

                batch = tokenizer(row, truncation=True, max_length=seq_length, return_tensors='pt')
                batch = {k: v.cuda() for k, v in batch.items()}

                out = gpt.forward(**batch,)
                loss = functional.cross_entropy(out.logits[:, :-1, :].flatten(0, -2), batch['input_ids'][:, 1:].flatten(),
                                      reduction='mean')
                perplexity = torch.exp(loss)

                if verbose:
                    print("Loss: ", loss)
                    print("Perplexity: ", perplexity)
                
                train_losses.append(loss)
                train_perplexities.append(perplexity)

                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            
            # Validation loop
            for row in tqdm(valid_data):
                if len(row) <= 1:
                    continue

                batch = tokenizer(row, truncation=True, max_length=seq_length, return_tensors='pt')
                batch = {k: v.cuda() for k, v in batch.items()}

                out = gpt.forward(**batch,)
                loss = functional.cross_entropy(out.logits[:, :-1, :].flatten(0, -2), batch['input_ids'][:, 1:].flatten(),
                                      reduction='mean')
                perplexity = torch.exp(loss)

                if verbose:
                    print("Loss: ", loss)
                    print("Perplexity: ", perplexity)
                
                valid_losses.append(loss)
                valid_perplexities.append(perplexity)

                if min_valid_loss > valid_loss:
                    min_valid_loss = valid_loss
                    # Saving State Dict
                    model_best_state = model.state_dict()

            if lr_update:
                scheduler.step()

    return model_best_state, losses, perplexities


best_state, loss, perplexity = train(train_data, 1, lr=1e-7, lr_update=True, weight_decay=0.01, verbose=True)

  0%|          | 0/5976 [00:00<?, ?it/s]

Loss:  tensor(6.2293, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(507.4171, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(3.2028, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(24.6014, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(3.8980, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(49.3032, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(2.8709, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(17.6535, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(4.3301, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(75.9508, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(5.7808, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(324.0237, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(3.2421, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity:  tensor(25.5881, device='cuda:0', grad_fn=<ExpBackward0>)
Loss:  tensor(3.0225, device='cuda:0', 

KeyboardInterrupt: ignored